# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [60]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [61]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-24-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [62]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


394


In [63]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

317


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [64]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [65]:
#Create a Keyspace

create_keyspace="CREATE KEYSPACE IF NOT EXISTS Sparkify_db \
WITH REPLICATION={'class': 'SimpleStrategy', 'replication_factor': 1}"

try:
    session.execute(create_keyspace)
except Exception as e:
    print(e)
#DESCRIBE keyspaces;

#### Set Keyspace

In [66]:
# Set KEYSPACE to the keyspace specified above
set_keyspace="USE Sparkify_db"

try:
    session.execute(set_keyspace)
except exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [67]:
##Analysis 1: Getting the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
## In order to complete analysis 1, session_id and session_item_num creates record unique, creating composite key of session_id and session_item_num as primary key.  
create_query="CREATE TABLE IF NOT EXISTS song_session_data (session_id int, session_item_num int, artist_name text, song_title text, song_length double, PRIMARY KEY (session_id, session_item_num))"

##create query
session.execute(create_query)
#select_query=""
select_query="SELECT artist_name, song_title, song_length FROM song_session_data WHERE session_id=%s AND session_item_num=%s"


In [68]:
###Assigning the file name 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO song_session_data (session_id, session_item_num, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [69]:
## Passing SELECT statement to result_query to verify the data was entered into the table

result_query=session.execute(select_query, (338, 4))
##Show data row from select_query 
for row in result_query:
    print(row)

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [70]:
## Analysis 2: to get the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
## In order to complete analysis 2, user_id, session_id creates record unique.
##Creating session_item_num as clustering key to create records unique because filter criteria requires to have only user_id and session_id.

create_query="CREATE TABLE IF NOT EXISTS song_session_data_by_user (user_id int, session_id int, session_item_num int, artist_name text, song_title text, \
user_f_name text, user_l_name text, PRIMARY KEY ((user_id, session_id), session_item_num))"
session.execute(create_query)

with open(file, encoding='utf8') as feedfile:
    csvreader=csv.reader(feedfile)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_session_data_by_user (user_id, session_id, session_item_num, artist_name, song_title, user_f_name, user_l_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assigning which column element should be assigned for each column in the INSERT statement as per screenshot.
        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        
##Analysis query 2
select_query="SELECT artist_name, song_title, user_f_name, user_l_name FROM song_session_data_by_user where user_id= %s and session_id= %s"

## Passing SELECT statement to result_query to verify the data was entered into the table
result_query=session.execute(select_query, (10, 182))
##Show data row from select_query
for row in result_query:
    print(row)
               


Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", user_f_name='Sylvie', user_l_name='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', user_f_name='Sylvie', user_l_name='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', user_f_name='Sylvie', user_l_name='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_f_name='Sylvie', user_l_name='Cruz')


In [78]:
##Analysis 3: Getting user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## In order to complete analysis 3, song_title is required as part of filtering criteria, but in order to make records unique user_id as clustering key will help.

create_query="CREATE TABLE IF NOT EXISTS song_title_data_by_user (song_title text, user_id int, user_f_name text, user_l_name text, PRIMARY KEY ((song_title), user_id))"
session.execute(create_query)

with open(file, encoding='utf8') as feedfile:
    csvreader=csv.reader(feedfile)
    next(csvreader)
    for line in csvreader:
        query = """INSERT INTO song_title_data_by_user (song_title, user_id, user_f_name, user_l_name)"""
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        
##Analysis 3 query
select_query="SELECT user_f_name, user_l_name FROM song_title_data_by_user where song_title= %s"

## Passing SELECT statement to result_query to verify the data was entered into the table
result_query=session.execute(select_query, ('All Hands Against His Own', ))

##Show data row from select_query
for row in result_query:
    print(row)


Row(user_f_name='Christian', user_l_name='Porter')


In [77]:
## This query is to check and confirm on query 3 - song_session_data_by_user , if data exists or not in the table.
##Validating Insert Action
result_query=session.execute("SELECT user_f_name, user_l_name FROM song_title_data_by_user where song_title='Rehab'")
                           
for row in result_query:
    print(row)


Row(user_f_name='Christian', user_l_name='Porter')


In [79]:
## This query is to check and confirm on query 1 - song_session_data , if data exists or not in the table.
##Validating Insert Action
result_query=session.execute("SELECT artist_name, song_title, song_length FROM song_session_data WHERE session_id=10 AND session_item_num=1")
                           
for row in result_query:
    print(row)

Row(artist_name='Rihanna', song_title='Rehab', song_length=293.82485)


### Drop the tables before closing out the sessions

In [1]:
## Droppig table before closing out the sessions

session.execute("Drop Table song_session_data")
session.execute("Drop Table song_title_data_by_user")
session.execute("Drop Table song_session_data_by_user")

NameError: name 'session' is not defined

### Close the session and cluster connection¶

In [2]:
session.shutdown()
cluster.shutdown()

NameError: name 'session' is not defined